In [9]:
from PIL import Image
import numpy as np
import random

In [36]:
def two_dim_to_greyscale(image: np.ndarray):
    """
    convert 2-dim np.array to 3-dim greyscale np.array
    param image: 2-dim np.array
    output greyscale: 3-dim np.array
    """
    height, width = image.shape[0], image.shape[1]
    greyscale = np.zeros(shape = [height, width, 3], dtype = np.uint8)
    greyscale[:,:,0] = image
    greyscale[:,:,1] = image
    greyscale[:,:,2] = image
    return greyscale

In [14]:
def rgb_to_greyscale(image: np.ndarray):
    """
    convert RGB image to greyscale
    param image: RGB image as np.ndarray
    output greyscale_image: image with 3 channels set to greyscale format
    """
    height, width = image.shape[0], image.shape[1]
    greyscale_image = np.zeros(shape = [height, width, 3], dtype = np.uint8)
    greyscale_image[:,:,0] = 0.2989*image[:,:,0] + 0.5870*image[:,:,1] + 0.1140*image[:,:,2]
    greyscale_image[:,:,1] = greyscale_image[:,:,0]
    greyscale_image[:,:,2] = greyscale_image[:,:,0]
    return greyscale_image

In [38]:
def normalize_image(image: np.ndarray):
    """
    linear normalization of image values to 0 - 255 range
    param image: image as np.array
    output image_normilized: normalized 'image'
    """
    min_value = np.min(image)
    max_value = np.max(image)
    image_normilized = np.floor(255.0 * (image - min_value) / (max_value - min_value))
    return image_normilized

In [39]:
#edge detection
def sobel(image: np.ndarray):
    """
    applies sobel operator to color intensities
    param image: greyscale image as 2-dimensional np.ndarray
    output result: non-normilized 2-dimensional greyscale image after applying sobel operator to 'image'
    """
    height, width = image.shape[0], image.shape[1]
    horizontal_mask = np.array([[-1, -2, -1],
                     [0, 0, 0],
                     [1, 2, 1]])
    vertical_mask = np.array([[-1, 0, 1],
                     [-2, 0, 2],
                     [-1, 0, 1]])
    result = np.zeros(shape = [height, width], dtype = np.uint8)
    for x in range(1,height-1):
        for y in range(1,width-1):
            g_x = np.sum(horizontal_mask * image[x-1:x+2, y-1:y+2])
            g_y = np.sum(vertical_mask * image[x-1:x+2, y-1:y+2])
            grad = np.sqrt(g_x**2+g_y**2)
            result[x,y] = grad
    result = normalize_image(result)
    return result

In [42]:
# adds salt and pepper noise to greyscaled image
def salt_and_pepper(image: np.ndarray, prob: float):
    """
    add random noise to image
    param 
    image: image as np.ndarray
    prob: half-probability of mutation
    output
    image: image with 'prob' portion of entries changed to 0 and 'prob' portion of entries changed to 255
    """
    height, width = image.shape[0], image.shape[1]
    threshold = 1 - prob
    random_matrix = np.random.random_sample((height, width))
    image[random_matrix < prob] = 0
    image[random_matrix > threshold] = 255
    return image

In [52]:
def median_filter(image: np.ndarray, filter_size = 1, mode = 'line'):
    height, width = image.shape[0], image.shape[1]
    result = np.zeros(shape = [height, width], dtype = np.uint8)
    for x in range(filter_size, height-filter_size):
        for y in range(filter_size, width-filter_size):
            if mode == 'square':
                result[x,y] = np.median(image[x-filter_size:x+filter_size+1,y-filter_size:y+filter_size+1])
            elif mode == 'line':
                result[x,y] = np.median(np.concatenate( (image[x-filter_size:x+filter_size+1,y],
                                                      image[x,y-filter_size:y],
                                                      image[x,y+1:y+filter_size+1]) ))
    return result

In [62]:
#edge detection
def first_order_approx(image: np.ndarray, normilize = True) -> np.ndarray:
    
    """
    edge detection by first order intensity approximation  f(i, j)= a*i + b*j + c

    param image: color intensity

    return: normilized gradient of first order approximation

    """
    
    height, width = image.shape[0], image.shape[1]
    result = np.zeros(shape=(height, width), dtype=np.uint8)
    a_mask = np.array([[-1, -1], [1, 1]])
    b_mask = np.array([[-1, 1], [-1, 1]])
    for i in range(1, height):
        for j in range(1, width):
            a = .5*np.sum(a_mask*image[i-1:i+1, j-1:j+1])
            b = .5*np.sum(b_mask * image[i-1:i+1, j-1:j+1])
            result[i,j] = np.sqrt(a**2+b**2)
    if (normilize):
        result = normalize_image(result)
    return result

In [57]:
def show_image_by_intensity(intensity: np.ndarray):
    height, width = intensity.shape[0], intensity.shape[1]
    image_arr = np.zeros(shape = [height, width, 3], dtype = np.uint8)
    image_arr[:,:,0] = intensity
    image_arr[:,:,1] = intensity
    image_arr[:,:,2] = intensity
    image = Image.fromarray(image_arr, 'RGB')
    image.show()

In [63]:
def main():
    image = Image.open(r'C:\Users\111\Desktop\Image_processing\photo.jpg')
    #image.show()
    image_arr = np.array(image)
    height, width = image_arr.shape[0], image_arr.shape[1]
    greyscale_image_arr = rgb_to_greyscale(image_arr)
    
    
    greyscale_image = Image.fromarray(greyscale_image_arr, 'RGB')
    greyscale_image.show()
    """
    mutated_intensity = salt_and_pepper(greyscale_image_arr[:,:,0], 0.1)
    show_image_by_intensity(mutated_intensity)
    
    
    filtered_line_intensity = median_filter(mutated_intensity, filter_size = 2, mode = 'line')
    show_image_by_intensity(filtered_line_intensity)
    
    filtered_square_intensity = median_filter(mutated_intensity, filter_size = 2, mode = 'square')
    show_image_by_intensity(filtered_square_intensity)
    """
    
    intensities_for_edge_detection_non_normilized = first_order_approx(greyscale_image_arr[:,:,0], normilize = False)
    show_image_by_intensity(intensities_for_edge_detection_non_normilized)
    intensities_for_edge_detection = normalize_image(intensities_for_edge_detection_non_normilized)
    show_image_by_intensity(intensities_for_edge_detection)
    """
    sobel_image_layer = sobel(greyscale_image_arr[:,:,0])
    sobel_image_arr = two_dim_to_greyscale(sobel_image_layer)
    sobel_image = Image.fromarray(sobel_image_arr, 'RGB')
    sobel_image.show()
    """
main()